In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader,random_split,Dataset
import torch.optim as optim
from tqdm import tqdm

In [2]:
trainset = datasets.CIFAR10(root='/home/aminul/data1/', train=True, download=False, transform=transforms.ToTensor())
testset = datasets.CIFAR10(root='/home/aminul/data1/', train=False, download=False, transform=transforms.ToTensor())

labels_list = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
non_animal = [0,1,8,9]
device = 'cuda:1'

In [3]:
class NewDataset(Dataset):
    
    def __init__(self,data,transform=None):
        self.data = data
        
    def __len__(self):
        return len(self.data)    
    
    def __getitem__(self,idx):
        image = self.data[idx][0]
        label1 = self.data[idx][1]          #original label 
        label2 = 0 if self.data[idx][1] in non_animal else 1       #animal or non-animal
        label3 = 0 if self.data[idx][1] > 5 else 1   #random labels
        return image, label1, label2, label3

In [4]:
new_trainset = NewDataset(trainset,non_animal)
new_testset = NewDataset(testset,non_animal)

train_loader = DataLoader(new_trainset, batch_size=100, shuffle=True)
test_loader = DataLoader(new_testset, batch_size=100, shuffle=True)

In [5]:
class Net(nn.Module):
    def __init__(self, input_channel, num_class):
        super(Net,self).__init__()
        
        self.classes = num_class
        
        self.conv1 = nn.Conv2d(in_channels=input_channel,out_channels=8,kernel_size=3,stride=1)
        self.conv2 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3,stride=1)
        self.fc1 = nn.Linear(64, 256)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256,128)
        self.dropout2 = nn.Dropout(0.3)
        
        self.fc3 = nn.Linear(128, self.classes[0])
        self.fc4 = nn.Linear(128, self.classes[1])
        self.fc5 = nn.Linear(128, self.classes[2])
        
    def forward(self, x):
        
        x = F.max_pool2d(F.relu(self.conv1(x)),kernel_size=3)
        x = F.max_pool2d(F.relu(self.conv2(x)),kernel_size=3)
        x = F.relu(self.fc1(x.reshape(-1,x.shape[1] * x.shape[2]*x.shape[3])))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x1 = self.fc3(x)
        x2 = self.fc4(x)
        x3 = self.fc5(x)
        
        return x1,x2,x3 

In [6]:
num_classes = [10,2,2]
net = Net(3,num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001,momentum=0.9, weight_decay=5e-4)

In [7]:
def train(net,trainloader,optim,criterion,epoch,device):
    net.train()
    train_loss,total,total_correct1,total_correct2,total_correct3 = 0,0,0,0,0
    
    for i,(inputs,tg1,tg2,tg3) in enumerate(tqdm(trainloader)):
        
        inputs,tg1,tg2,tg3 = inputs.to(device), tg1.to(device), tg2.to(device), tg3.to(device)
        optim.zero_grad()
        
        op1,op2,op3 = net(inputs)
        loss1 = criterion(op1,tg1)
        loss2 = criterion(op2,tg2)
        loss3 = criterion(op3,tg3)
        
        loss1.backward(retain_graph=True)
        loss2.backward(retain_graph=True)
        loss3.backward()
        
        optim.step()
        
        train_loss += loss1.item() + loss2.item() + loss3.item()
        _,pd1 = torch.max(op1.data,1)
        _,pd2 = torch.max(op2.data,1)
        _,pd3 = torch.max(op3.data,1)
        total_correct1 += (pd1 == tg1).sum().item()
        total_correct2 += (pd2 == tg2).sum().item()
        total_correct3 += (pd3 == tg3).sum().item()
        total += tg1.size(0)
    
    print("Epoch: [{}]  loss: [{:.2f}] Orig_Acc [{:.2f}] animal_Acc [{:.2f}] random_Acc [{:.2f}] ".format
                                                                          (epoch+1,train_loss/(i+1),
                                                                           (total_correct1*100/total),
                                                                          (total_correct2*100/total)
                                                                         ,(total_correct3*100/total)))
    return train_loss/(i+1)

In [8]:
def tester(net,testloader,optim,criterion,epoch,device):
    net.eval()
    test_loss,total,total_correct1,total_correct2,total_correct3 = 0,0,0,0,0
    
    for i,(inputs,tg1,tg2,tg3) in enumerate(tqdm(testloader)):
        
        inputs,tg1,tg2,tg3 = inputs.to(device), tg1.to(device), tg2.to(device), tg3.to(device)
        
        op1,op2,op3 = net(inputs)
        loss1 = criterion(op1,tg1)
        loss2 = criterion(op2,tg2)
        loss3 = criterion(op3,tg3)
        
        test_loss += loss1.item() + loss2.item() + loss3.item()
        _,pd1 = torch.max(op1.data,1)
        _,pd2 = torch.max(op2.data,1)
        _,pd3 = torch.max(op3.data,1)
        total_correct1 += (pd1 == tg1).sum().item()
        total_correct2 += (pd2 == tg2).sum().item()
        total_correct3 += (pd3 == tg3).sum().item()
        total += tg1.size(0)
        
    acc1 = 100. * total_correct1 / total
    acc2 = 100. * total_correct2 / total
    acc3 = 100. * total_correct3 / total
    print("\nTest Epoch #%d Loss: %.4f Orig_Acc: %.2f%% animal_Acc: %.2f%% random_Acc: %.2f%%" %(epoch+1,
                                                                                                  test_loss/(i+1),
                                                                                                  acc1,acc2,acc3))
        
    return test_loss/(i+1), acc1, acc2, acc3

In [9]:
num_epochs = 20
train_loss,test_loss = [],[]

for epoch in range(num_epochs):
    
    a = train(net,train_loader,optimizer,criterion,epoch,device)
    c,_,_,_ = tester(net,test_loader,optimizer,criterion,epoch,device)    
    
    train_loss.append(a), test_loss.append(c)

  2%|▏         | 2/100 [00:00<00:06, 14.54it/s]

Epoch: [1]  loss: [3.65] Orig_Acc [9.97] animal_Acc [59.87] random_Acc [60.00] 


  0%|          | 2/500 [00:00<00:35, 14.21it/s]


Test Epoch #1 Loss: 3.6406 Orig_Acc: 10.49% animal_Acc: 60.00% random_Acc: 60.00%


  2%|▏         | 2/100 [00:00<00:05, 16.78it/s]

...


Test Epoch #19 Loss: 2.4188 Orig_Acc: 41.71% animal_Acc: 89.43% random_Acc: 70.15%


  2%|▏         | 2/100 [00:00<00:06, 14.46it/s]

Epoch: [20]  loss: [2.50] Orig_Acc [39.90] animal_Acc [88.75] random_Acc [67.35] 


100%|██████████| 100/100 [00:05<00:00, 16.85it/s]


Test Epoch #20 Loss: 2.4025 Orig_Acc: 42.32% animal_Acc: 89.41% random_Acc: 69.88%
